In [0]:
from pyspark.sql.functions import *

In [0]:
# read data from source
df = spark.read.format("parquet").load("abfss://bronze@databricksdevetl.dfs.core.windows.net/products")
df.display()

In [0]:
df = df.drop("order_id", "customer_id", "order_date", "quantity", "total_amount", "first_name", "last_name", "email", "city", "state", "_rescued_data")
df.display()

In [0]:
df.createOrReplaceTempView("products")

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_dev.bronze.discount_udf(p_price double) 
RETURNS double
LANGUAGE SQL
RETURN ROUND((p_price * 0.90), 2)

In [0]:
%sql
SELECT product_id, databricks_dev.bronze.discount_udf(price) as discounted_price FROM products

In [0]:
df = df.withColumn("discounted_price", expr("databricks_dev.bronze.discount_udf(price)"))
df.display()

In [0]:
# write date to silver layer
df.write.mode("overwrite") \
    .format("delta") \
    .save("abfss://silver@databricksdevetl.dfs.core.windows.net/products")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_dev.silver.silver_products
USING DELTA
LOCATION "abfss://silver@databricksdevetl.dfs.core.windows.net/products"